In [19]:
from dotenv import load_dotenv
from langchain.chains import create_history_aware_retriever, create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.document_loaders.csv_loader import CSVLoader
from langchain_chroma import Chroma
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [20]:
load_dotenv()

True

# Defining the Model

In [21]:
model = ChatOpenAI(model="gpt-4o", temperature=0.15)

# Retriever

In [22]:
loader = CSVLoader('context.csv')
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())

retriever = vectorstore.as_retriever()

# Building the system

### Contextualizing the user question

In [36]:
contextualization_system_prompt = """
    Dado um histórico de chat e a última pergunta do usuário que pode fazer referência ao contexto no histórico de chat, 
    formule uma pergunta autônoma que possa ser entendida sem o histórico de chat. NÃO responda à pergunta, 
    apenas reformule-a se necessário e, caso contrário, retorne-a como está.
"""

In [37]:
contextualization_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualization_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

In [38]:
history_aware_retriever = create_history_aware_retriever(
    model, retriever, contextualization_prompt
)

### Answering the question

In [39]:
system_prompt = """Você é um chatbot para atendimento e vendas aos clientes de uma empresa de eletrônicos, a Gabs LTDA.

Os clientes irão te fazer perguntas sobre os produtos e serviços da empresa.
Responda as perguntas dos clientes apenas usando os dados fornecidos.
Se você não souber, diga "Não sei, por favor entre em contato com o nosso suporte ao cliente".
Use uma linguagem envolvente, cortês e profissional semelhante à de um representante de atendimento ao cliente.

Responda as perguntas na intenção de finalizar uma compra, mas não seja insistente. 
Capture o nome, email e telefone dos clientes que forem comprar algum produto e os agradeça pela compra.
Ao capturar os dados necessários, lembre-se que nem todos usuários estão familiarizados com o processo de compra online.
Se for necessário peça os dados um por um.

Ao finalizar uma compra bem sucedida mostre ao cliente a URL de checkout: 'https://www.youtube.com/watch?v=dQw4w9WgXcQ'.
Ao finalizar uma compra mal sucedida, se despessa brevemente e agradeça a conversa.

Caso o cliente pergunte sobre um produto que não está disponível na base de dados, informe que o produto está em falta e ofereça um produto similar.

{context}
"""

In [40]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

In [41]:
question_answer_chain = create_stuff_documents_chain(model, prompt)

In [42]:
rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)

### Managing chat history (by session_id)

In [43]:
store = {}

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

In [44]:
conversational_rag_chain = RunnableWithMessageHistory(
    rag_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
    output_messages_key="answer",
)

# Running the system

In [45]:
conversational_rag_chain.invoke(
    {"input": "Quais são os produtos disponíveis?"},
    config={
        "configurable": {"session_id": "abc123"}
    },
)["answer"]

'Vendemos uma variedade de produtos eletrônicos, incluindo:\n\n- Smartphones\n- Laptops\n- Smartwatches\n- Tablets\n- Fones de Ouvido\n- Câmeras\n- Smart TVs\n- Consoles de Videogame\n\nSe precisar de mais informações sobre algum produto específico, estou à disposição para ajudar!'

In [33]:
conversational_rag_chain.invoke(
    {"input": "Quais são os smartphones disponíveis?"},
    config={
        "configurable": {"session_id": "abc123"}
    },
)["answer"]

'Temos disponível o NovoPhone X12 por R$ 3499. Gostaria de saber mais sobre ele ou está interessado em realizar a compra? Estou aqui para ajudar!'

In [34]:
conversational_rag_chain.invoke(
    {"input": "Quais são os smartwatches disponíveis?"},
    config={
        "configurable": {"session_id": "def456"}
    },
)["answer"]

'Temos disponível o TimeWatch S2 por R$ 1299. \n\nGostaria de saber mais sobre ele ou realizar a compra?'

In [35]:
conversational_rag_chain.invoke(
    {"input": "Eu gostei do NovoPhone X12, como faço para comprar?"},
    config={
        "configurable": {"session_id": "abc123"}
    },
)["answer"]

'Que ótimo saber que você gostou do NovoPhone X12! Para prosseguirmos com a compra, vou precisar de algumas informações suas. Vamos começar com o seu nome, por favor?'